In [1]:
import pandas as pd
import os
import cv2
import numpy as np
from numpy import save
from keras.models import load_model

2021-12-16 10:55:59.404159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/eduardo/anaconda3/envs/Project/lib/python3.9/site-packages/cv2/../../lib64:
2021-12-16 10:55:59.404313: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df=pd.read_csv('full_df.csv')

In [3]:
df.iloc[665]

ID                                                                         766
Patient Age                                                                 64
Patient Sex                                                               Male
Left-Fundus                                                       766_left.jpg
Right-Fundus                                                     766_right.jpg
Left-Diagnostic Keywords                                         normal fundus
Right-Diagnostic Keywords                          macular epiretinal membrane
N                                                                            0
D                                                                            0
G                                                                            0
C                                                                            0
A                                                                            0
H                                                   

In [290]:
df[df.filename=='4_left.jpg']

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,...,A,H,M,O,filepath,labels,target,filename,age_bin,Age_Types_Cat
3197,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,...,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['O'],"[0, 0, 0, 0, 0, 0, 0, 1]",4_left.jpg,46-55,3


In [16]:
c=df.groupby(['Patient Age'])['Patient Age'].sum().unstack

In [17]:
c

<bound method Series.unstack of Patient Age
1       28
14      28
15      30
17      34
18      36
      ... 
86     172
87    1305
88     176
89     356
91     182
Name: Patient Age, Length: 75, dtype: int64>

In [90]:
def cutAge(dataframe: pd.DataFrame, age: str) -> pd.Categorical:

    bins = [0, 18, 35,45,55,60, 65, 80, 120]
    labels = ['1-18', '19-35', '36-45','46-55','56-60','61-65', '66-80', '81+']
    ageBins = pd.cut(dataframe[age], bins = bins , labels = labels)
    
    return ageBins

In [91]:
df['age_bin'] = cutAge(df, 'Patient Age')

In [92]:
df['age_bin'].value_counts()

46-55    1594
66-80    1486
56-60    1189
61-65    1131
36-45     622
19-35     219
81+       115
1-18       36
Name: age_bin, dtype: int64

In [97]:
c=df['Age_Types_Cat'].values

In [98]:
d=df['filename'].values

In [283]:
len(d)

6392

In [95]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
bin_types = df.age_bin.unique()
labelencoder = LabelEncoder()
df['Age_Types_Cat'] = labelencoder.fit_transform(df['age_bin'])


In [96]:
df.Age_Types_Cat.value_counts()

3    1594
6    1486
4    1189
5    1131
2     622
1     219
7     115
0      36
Name: Age_Types_Cat, dtype: int64

In [190]:
def create_dataset(image_category):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for i in range(len(image_category)):
        image_path = os.path.join(dataset_dir,image_category[i])
        img = cv2.imread(image_path,0)
        img = cv2.resize(img, (250,250))
        img = img.astype(np.float32)/255
        img = img[..., np.newaxis]
        l=df3.iloc[i][['Patient Age']]        
        dataset.append([np.array(img),np.array(l)])

    return dataset

In [192]:
Age=create_dataset(d)

In [196]:
Age[0][1]

array([69], dtype=object)

In [ ]:
def create_dataset2(image_category):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for i in range(len(image_category)):
        image_path = os.path.join(dataset_dir,image_category[i])
        img = cv2.imread(image_path,0)
        img = cv2.resize(img, (250,250))
        img = img.astype(np.float32)/255
        img = img[..., np.newaxis]
        l=df3.iloc[i][['Age_Types_Cat_0', 'Age_Types_Cat_1', 'Age_Types_Cat_2','Age_Types_Cat_3','Age_Types_Cat_4', 'Age_Types_Cat_5','Age_Types_Cat_6','Age_Types_Cat_7']]        
        dataset.append([np.array(img),np.array(l)])

    return dataset

In [292]:
df.iloc[2].filename

'2_right.jpg'

In [176]:
from keras.preprocessing.image import ImageDataGenerator
def augment2(num,mult):
    x=[]
    df2=df3[df3[f'Age_Types_Cat_{num}']==1]
    category=df2.filename.values
    for img in category:
        label=df3.iloc[df2[df2.filename==img].index[0]][['Age_Types_Cat_0', 'Age_Types_Cat_1', 'Age_Types_Cat_2','Age_Types_Cat_3','Age_Types_Cat_4', 'Age_Types_Cat_5','Age_Types_Cat_6','Age_Types_Cat_7']]   
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",0)
            img = cv2.resize(img, (250,250))
            img = img[..., np.newaxis]
            img = img.astype(np.float32)/255
            
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=20,
                    zoom_range=0.2,
                    brightness_range=(0.4, 1.2),
                    height_shift_range=0.2,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
        
        
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [193]:
from keras.preprocessing.image import ImageDataGenerator
def augment3(num,mult):
    x=[]
    df2=df3[df3[f'Age_Types_Cat_{num}']==1]
    category=df2.filename.values
    for img in category:
        label=df3.iloc[df2[df2.filename==img].index[0]][['Patient Age']]   
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",0)
            img = cv2.resize(img, (250,250))
            img = img[..., np.newaxis]
            img = img.astype(np.float32)/255
            
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=20,
                    zoom_range=0.2,
                    brightness_range=(0.4, 1.2),
                    height_shift_range=0.2,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
        
        
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [197]:
Aug0=augment3(0,50)

In [200]:
len(Aug0)

1800

In [208]:
Aug1=augment3(1,5)

In [209]:
len(Aug1)

1095

In [210]:
Aug7=augment3(7,12)

In [211]:
len(Aug7)

1380

In [212]:
Aug2=augment3(2,2)

In [213]:
len(Aug2)

1244

In [276]:
random.shuffle(Age)

In [277]:
AgeDef=Age[:-1000]
AgePrueba=Age[-1000:]

In [278]:
Todo2=AgeDef+Aug1+Aug7+Aug2+Aug0

In [279]:
len(Todo2)

10911

In [243]:
Lista=[1,2,3,4,5,6]

In [244]:
Lista2=random.shuffle(Lista)

In [246]:
type(Lista2)

NoneType

In [247]:
random.shuffle(Todo)

In [252]:
len(Todo)

11911

In [253]:
len(TodoDef)

10911

In [254]:
len(TodoPrueba)

1000

In [248]:
TodoDef=Todo[:-1000]

In [250]:
TodoPrueba=Todo[-1000:]

In [284]:
x = np.array([i[0] for i in AgePrueba])
y = np.array([i[1] for i in AgePrueba]).astype(int)

In [285]:
model=load_model("my_model_AgePrediction3.h5")

In [286]:
y_pred=model.predict(x)

In [287]:
y_pred

array([[59.36155 ],
       [52.688004],
       [52.17987 ],
       [58.74225 ],
       [52.537285],
       [77.90828 ],
       [56.814735],
       [62.153183],
       [64.06695 ],
       [66.873184],
       [45.193047],
       [56.1093  ],
       [53.332035],
       [57.733414],
       [54.218956],
       [40.490196],
       [54.39657 ],
       [36.520607],
       [72.637024],
       [66.198715],
       [44.335228],
       [55.544792],
       [53.744873],
       [51.61502 ],
       [26.658512],
       [59.519478],
       [53.91023 ],
       [66.98956 ],
       [52.373775],
       [61.03209 ],
       [61.931847],
       [55.469128],
       [54.44831 ],
       [59.723248],
       [57.35359 ],
       [37.884518],
       [60.897026],
       [70.7699  ],
       [34.288063],
       [51.56104 ],
       [43.67121 ],
       [39.862   ],
       [56.020554],
       [58.482037],
       [63.39687 ],
       [61.133118],
       [51.214073],
       [61.855427],
       [37.52848 ],
       [50.503395],


In [271]:
from sklearn.metrics import r2_score,mean_absolute_error

In [288]:
print(r2_score(y,y_pred))
print(mean_absolute_error(y,y_pred))

0.35546879397628206
6.97274374961853


In [281]:
save('x5',x)

In [282]:
save('y5',y)

In [150]:
df3.iloc[df2[df2.filename==c].index[0]][['Age_Types_Cat_0', 'Age_Types_Cat_1', 'Age_Types_Cat_2','Age_Types_Cat_3','Age_Types_Cat_4', 'Age_Types_Cat_5','Age_Types_Cat_6','Age_Types_Cat_7']]

Age_Types_Cat_0    1
Age_Types_Cat_1    0
Age_Types_Cat_2    0
Age_Types_Cat_3    0
Age_Types_Cat_4    0
Age_Types_Cat_5    0
Age_Types_Cat_6    0
Age_Types_Cat_7    0
Name: 1013, dtype: object

In [127]:
Age=create_dataset(d,c)

In [131]:
Age[0][1]

array([0, 0, 0, 0, 0, 0, 1, 0], dtype=object)

In [102]:
x = np.array([i[0] for i in Age])
y = np.array([i[1] for i in Age])

In [116]:
from sklearn.preprocessing import OneHotEncoder
y = np.array(df3.iloc[:][['Age_Types_Cat_0', 'Age_Types_Cat_1', 'Age_Types_Cat_2','Age_Types_Cat_3','Age_Types_Cat_4', 'Age_Types_Cat_5','Age_Types_Cat_6','Age_Types_Cat_7']])
y = np.array(y)
print('Label :   '+str(y.shape))

Label :   (6392, 8)


In [125]:
y[6]

array([0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8)

In [105]:
save('x3',x)

In [118]:
save('y4',y)